# **1. SILENT INSTALLATION (compatible versions)**

In [11]:
print("Setting up Persian Medical RAG Agent...")

import subprocess
import sys

def silent_install():
    packages = [
        "numpy>=1.26.0,<2.0.0",          # Compatible with langchain-huggingface 0.0.3
        "sentence-transformers",          # Required for embeddings
        "chromadb==0.5.3",
        "langchain==0.2.16",
        "langchain-community==0.2.16",
        "langchain-huggingface==0.0.3",
        "langchain-groq==0.1.9",
        "langgraph==0.2.5",
        "tavily-python==0.4.0",
        "nest_asyncio"
    ]
    for pkg in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

silent_install()
print("✅ Setup complete!")

Setting up Persian Medical RAG Agent...
✅ Setup complete!


# **2. IMPORTS**

In [12]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["USER_AGENT"] = "PersianMedicalRAG/2.0"  # Updated version

import nest_asyncio
from IPython.display import Markdown, display
from typing import List, Literal, TypedDict

# LangChain imports
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import BaseMessage
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, END

nest_asyncio.apply()

# **3. CONFIGURATION**

In [13]:
class Config:
    GROQ_API_KEY   = "GROQ_API_KEY"
    TAVILY_API_KEY = "TAVILY_API_KEY"
    LLM_MODEL      = "llama-3.3-70b-versatile"  # ✅ New model
    EMBEDDING_MODEL = "HooshvareLab/bert-fa-base-uncased"

config = Config()
os.environ["GROQ_API_KEY"]   = config.GROQ_API_KEY
os.environ["TAVILY_API_KEY"] = config.TAVILY_API_KEY
print("✅ APIs configured | LLM Ready (llama-3.3-70b-versatile)")

✅ APIs configured | LLM Ready (llama-3.3-70b-versatile)


# **4. KNOWLEDGE BASE (Persian medical pages)**

In [14]:
print("Building medical knowledge base...")

embedding_model = HuggingFaceEmbeddings(
    model_name=config.EMBEDDING_MODEL,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

urls = [
    "https://fa.wikipedia.org/wiki/مالاریا",
    "https://fa.wikipedia.org/wiki/دیابت_قندی",
    "https://fa.wikipedia.org/wiki/میگرن",
    "https://www.darmankade.com/blog/malaria-symptoms/",
    "https://www.darmankade.com/blog/diabetes-type-1/",
    "https://www.darmankade.com/blog/migraine/",
]

print("Loading documents...")
loader = WebBaseLoader(urls)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

print("Creating vector store...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    collection_name="fa_medical"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print(f"✅ Knowledge base ready: {len(chunks)} chunks")

Building medical knowledge base...
Loading documents...
Creating vector store...
✅ Knowledge base ready: 628 chunks


# **5. LLM & WEB SEARCH**

In [15]:
llm = ChatGroq(model=config.LLM_MODEL, temperature=0.0)
tavily_search = TavilySearchResults(max_results=3)

def web_search(query: str) -> List[Document]:
    """Return Tavily results as Document objects."""
    try:
        results = tavily_search.invoke(query)
        return [Document(page_content=r["content"], metadata={"url": r["url"]}) for r in results]
    except Exception:
        return []

# **6. ROUTING & RELEVANCE**

In [16]:
class RouteQuery(BaseModel):
    route: Literal["vectorstore", "websearch", "fallback"]

router_prompt = ChatPromptTemplate.from_template(
    """شما یک مسیریاب هوشمند هستید. سؤال فارسی را بررسی کنید:
- اگر درباره مالاریا، دیابت (نوع ۱ یا ۲) یا میگرن → "vectorstore"
- اگر پزشکی دیگر → "websearch"
- اگر غیرپزشکی → "fallback"

سؤال: {query}
Route:"""
)
router = router_prompt | llm.with_structured_output(RouteQuery)

class RelevanceGrader(BaseModel):
    grade: Literal["relevant", "irrelevant"]

grader_prompt = ChatPromptTemplate.from_messages([
    ("system", "مرتبط بودن را درجه‌بندی کنید: 'relevant' یا 'irrelevant' فقط"),
    ("human", "سؤال: {query}\nمتن: {context}")
])
grader = grader_prompt | llm.with_structured_output(RelevanceGrader)

# **7. RESPONSE CHAINS**

In [17]:
rag_prompt = ChatPromptTemplate.from_template(
    """شما یک دستیار پزشکی فارسی هستید. فقط بر اساس متن زیر پاسخ دهید.

متن:
{context}

سؤال: {query}
پاسخ به فارسی:"""
)
rag_chain = (
    {
        "context": lambda x: "\n\n".join([d.page_content for d in x["docs"]]),
        "query": lambda x: x["query"]
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

fallback_prompt = ChatPromptTemplate.from_template(
    """شما یک دستیار پزشکی فارسی هستید.
به سؤال مناسب پاسخ دهید (به فارسی).
سؤال: {query}
پاسخ:"""
)
fallback_chain = fallback_prompt | llm | StrOutputParser()

# **8. LANGGRAPH WORKFLOW**

In [18]:
class AgentState(TypedDict):
    query: str
    chat_history: List[BaseMessage]
    docs: List[Document]
    generation: str

def retrieve(state: AgentState):
    docs = retriever.invoke(state["query"])
    return {"docs": docs}

def web_search_node(state: AgentState):
    docs = web_search(state["query"])
    return {"docs": docs}

def filter_docs(state: AgentState):
    filtered = []
    for doc in state["docs"]:
        try:
            grade = grader.invoke({"query": state["query"], "context": doc.page_content})
            if grade.grade == "relevant":
                filtered.append(doc)
        except Exception:
            continue
    return {"docs": filtered}

def generate_answer(state: AgentState):
    if state["docs"]:
        generation = rag_chain.invoke({"docs": state["docs"], "query": state["query"]})
    else:
        generation = "اطلاعات پزشکی مرتبط یافت نشد."
    return {"generation": generation}

def generate_fallback(state: AgentState):
    generation = fallback_chain.invoke({"query": state["query"]})
    return {"generation": generation}

# **9. BUILD GRAPH**

In [19]:
print("Building AI agent...")
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve)
workflow.add_node("websearch", web_search_node)
workflow.add_node("filter", filter_docs)
workflow.add_node("rag", generate_answer)
workflow.add_node("fallback", generate_fallback)

def route_question(state: AgentState):
    result = router.invoke({"query": state["query"]})
    return result.route

workflow.set_conditional_entry_point(
    route_question,
    {
        "vectorstore": "retrieve",
        "websearch":   "websearch",
        "fallback":    "fallback",
    },
)

workflow.add_edge("retrieve", "filter")
workflow.add_edge("websearch", "filter")
workflow.add_edge("fallback", END)

workflow.add_conditional_edges(
    "filter",
    lambda s: "rag" if s["docs"] else "websearch",
    {"rag": "rag", "websearch": "websearch"}
)

workflow.add_edge("rag", END)

medical_agent = workflow.compile()
print("✅ Medical RAG Agent ready (with llama-3.3-70b-versatile)!")

Building AI agent...
✅ Medical RAG Agent ready (with llama-3.3-70b-versatile)!


# **10. USER INTERFACE**

In [20]:
def ask(question: str) -> str:
    try:
        result = medical_agent.invoke({"query": question, "chat_history": []})
        return result.get("generation", "پاسخی تولید نشد.")
    except Exception as e:
        return f"خطا: {str(e)}"

def demo():
    samples = [
        ("علائم مالاریا چیست؟", "VectorStore - Malaria"),
        ("درمان دیابت نوع ۲ چگونه است؟", "VectorStore - Diabetes"),
        ("میگرن چه علائمی دارد؟", "VectorStore - Migraine"),
        ("فشار خون بالا چه خطراتی دارد؟", "Web Search"),
        ("سلام چطوری؟", "Fallback")
    ]
    print("\n" + " DEMONSTRATION ".center(60, "="))
    for q, cat in samples:
        print(f"\n[{cat}]")
        print(f"سؤال: {q}")
        print("-" * 50)
        answer = ask(q)
        display(Markdown(f"**پاسخ:** {answer}"))
    print("="*60)

def chat():
    print("\nInteractive Medical Chat")
    print("Type 'exit' or 'خروج' to quit, 'demo' for examples\n")
    while True:
        user = input("شما: ").strip()
        if user.lower() in ["exit", "خروج", "quit"]:
            print("ممنون از استفاده!")
            break
        if user.lower() == "demo":
            demo()
            continue
        if not user:
            continue
        print("در حال پردازش...")
        resp = ask(user)
        display(Markdown(f"**دستیار:** {resp}"))

# **11. QUICK START**

In [21]:
if __name__ == "__main__":
    print("\n" + " PERSIAN MEDICAL RAG AGENT v2 ".center(60, "="))
    print("تخصص: مالاریا، دیابت، میگرن")
    print("عمومی: سایر موضوعات پزشکی از طریق جستجوی وب")
    print("مدل: llama-3.3-70b-versatile (به‌روز‌شده)")
    print("="*60)
    
    demo()      # Run demo
    chat()      # Start interactive mode


=============== PERSIAN MEDICAL RAG AGENT v2 ===============
تخصص: مالاریا، دیابت، میگرن
عمومی: سایر موضوعات پزشکی از طریق جستجوی وب
مدل: llama-3.3-70b-versatile (به‌روز‌شده)

====================== DEMONSTRATION =======================

[VectorStore - Malaria]
سؤال: علائم مالاریا چیست؟
--------------------------------------------------


**پاسخ:** علائم مالاریا عبارتند از: تب 38 درجه سانتیگراد یا بیشتر، احساس تب و لرز، سردرد، استفراغ، دردهای عضلانی، اسهال. همچنین ممکن است باعث کم خونی و زردی پوست و چشم شود.


[VectorStore - Diabetes]
سؤال: درمان دیابت نوع ۲ چگونه است؟
--------------------------------------------------


**پاسخ:** درمان دیابت نوع ۲ شامل تغییرات در رژیم غذایی، افزایش فعالیت فیزیکی و در برخی موارد، استفاده از داروهای خاص است.


[VectorStore - Migraine]
سؤال: میگرن چه علائمی دارد؟
--------------------------------------------------


...

شما:  علت محبوبیت زبان پایتون چیست؟


در حال پردازش...


**دستیار:** زبان برنامه‌نویسی پایتون به دلیل سادگی، انعطاف‌پذیری و کارایی بالا در بسیاری از زمینه‌ها از جمله علم داده، یادگیری ماشین و توسعه وب بسیار محبوب است.

شما:  خروج


ممنون از استفاده!
